In [8]:
import requests
import bs4
import json

url = "https://www.alphavantage.co/documentation/"

response = requests.get(url)
if response.status_code != 200:
    raise RuntimeError("Failed to get HTML documentation from AlphaVantage!")

soup = bs4.BeautifulSoup(response.text, "html.parser")

In [ ]:
def process_section(
    section: bs4.element.Tag,
) -> tuple[str, dict[str, dict[str, str | list[str]]]]:
    section_title = section.find("h2").text
    # print("Section Title:", section_title)

    collection = {}
    contents = [
        c for c in section.contents if c != "\n" and not str(c).startswith("<br/")
    ]
    line_cnt = 0
    for k in range(1000):
        try:
            while not str(contents[line_cnt]).startswith("<h4"):
                line_cnt += 1
        except IndexError:
            break
        func_name = contents[line_cnt].text
        id_ = contents[line_cnt].get("id")
        if func_name.startswith("Quote Endpoint"):
            func_name = "GLOBAL_QUOTE"

        line_cnt += 1
        descr = []
        while str(contents[line_cnt]).startswith("<p"):
            if contents[line_cnt].text != "":
                descr.append(contents[line_cnt].text.strip())
            line_cnt += 1
        description = "\n".join(descr)

        assert str(contents[line_cnt]) == "<h6><b>API Parameters</b></h6>"
        line_cnt += 1
        reqs = []
        opts = []
        for j in range(100):
            if (
                str(contents[line_cnt])
                == "<p><b>❚ Required: <code>apikey</code></b></p>"
            ):
                line_cnt += 2
                break
            argument = str(contents[line_cnt].find("code").text)
            if argument == "function":
                code = contents[line_cnt + 1].find("code")
                if code is not None:
                    func_name = code.text.split("=")[1]
                line_cnt += 2
                continue

            lines = []
            is_req = str(contents[line_cnt]).startswith("<p><b>❚ Required: ")
            line_cnt += 1
            while "❚" not in str(contents[line_cnt]):
                lines.append(contents[line_cnt].text.strip())
                line_cnt += 1

            annotated_content = [argument, "\n".join(lines)]
            if is_req:
                reqs.append(annotated_content)
            else:
                opts.append(annotated_content)
        else:
            raise RuntimeError("Infinite Inner Section Loop Detected")

        assert contents[line_cnt].text.startswith("Example")
        line_cnt += 1

        examples = []
        for x in range(100):
            # TODO: We have an infinite loop here
            if str(contents[line_cnt]).startswith('<div class="premium-msg">'):
                break
            if isinstance(contents[line_cnt], bs4.element.Tag):
                if contents[line_cnt].find("a") is None:
                    example_descr = contents[line_cnt].text.strip()
                    line_cnt += 1
                else:
                    example_descr = "NO_DESCRIPTION"
            else:
                assert isinstance(contents[line_cnt], bs4.element.NavigableString)
                example_descr = contents[line_cnt].text.strip()
                line_cnt += 1
            urls = []
            try:
                while (
                    (not str(contents[line_cnt]).startswith('<div class="premium-msg">'))
                    and isinstance(contents[line_cnt], bs4.element.Tag)
                    and (contents[line_cnt].find("a") is not None)
                ):
                    urls.append(contents[line_cnt].find("a")["href"])
                    line_cnt += 1
            except IndexError:
                print(1)
            example_urls = "\n".join(urls)
            examples.append([example_descr, example_urls])
        else:
            raise RuntimeError("Inifinite Inner Section 2 Loop Detected")
        line_cnt += 1

        assert contents[line_cnt].text == "Language-specific guides"
        line_cnt += 1
        assert str(contents[line_cnt]).startswith("<div>\n<button class=")

        assert func_name is not None
        collection[func_name] = {
            "description": description,
            "id": id_,
            "examples": examples,
            "args_required": reqs,
            "args_optional": opts,
        }
        func_name = None
    else:
        raise RuntimeError("Infinite Outer Section Loop Detected")
    return section_title, collection


sections = soup.find_all("section")
section_dict = {}
for section in sections:
    section_title, collection = process_section(section)
    if section_title == "Digital & Crypto Currencies":
        del collection[
            "CURRENCY_EXCHANGE_RATE"
        ]  # This is duplicated, also appears in FX
    section_dict[section_title] = collection
    break

1


IndexError: list index out of range

In [ ]:
import io
import sys

# Catch print statements into memory
output_stream = io.StringIO()

original_stdout = sys.stdout
sys.stdout = output_stream

def format_opt_request_arg(arg: str) -> str:
    return f'([f"{arg}={LBRACE}{arg}{RBRACE}"] if {arg} is not None else [])'

def format_opt_arg(arg: str) -> str:
    return f"{arg}:Optional[any]=None" 


try:
    LBRACE = "{"
    RBRACE = "}"

    for section, dict_ in section_dict.items():
        print("    ", "#" * (len(section) + 4), sep = "")
        print("    ", "# ", section, " #", sep = "")
        print("    ", "#" * (len(section) + 4), sep = "")

        print()
        for k, v in dict_.items():
            args_req = [a[0] for a in v["args_required"]]
            args_req_str = ", ".join(args_req)
            args_req_request = [f'"{arg}={arg}"' for arg in args_req]

            args_opt = [a[0] for a in v["args_optional"]]
            args_opt_adj = [format_opt_arg(arg) for arg in args_opt]
            args_opt_str = ", ".join(args_opt_adj)
            args_opt_request = [format_opt_request_arg(arg) for arg in args_opt]

            args = ["self"]
            if args_req_str != "":
                args.append(args_req_str)
            if args_opt_str != "":
                args.append(args_opt_str)
            print("    ", f"def get_{k.lower()}({','.join(args)},**kwargs) -> dict[str, any]:", sep="")
            print('        """')
            print(f"https://www.alphavantage.co/documentation/#{v['id']}")
            for line_cnt in v["description"].splitlines():
                print("", line_cnt, sep = "")
            for arg, desc in v["args_required"]:
                print(f"### {arg} (required)")
                print(f"{desc}")
            for arg, desc in v["args_optional"]:
                print(f"### {arg} (optional)")
                print(f"{desc}")
            print('        """')
            request_args_optional = f" + {' + '.join(args_opt_request)}" if len(args_opt_request) > 0 else ""
            print(f"""
        return self._send_request(
            function="{k}",
            request_args=[{','.join(args_req_request)}]{request_args_optional},
            **kwargs
        )
            """)
finally:
    sys.stdout = original_stdout

python_code = output_stream.getvalue()

with open("util/_av_integration_api_base.py", "r") as file:
    code_base = file.read()
with open("testing.py", "w") as file:
    file.write(code_base + "\n" + python_code)